In [2]:
import torch
print(torch)
torch.tensor([1, 2, 3])


<module 'torch' from '/home/vishnu/Karpathy_Zero_To_Hero_Series/venv/lib/python3.12/site-packages/torch/__init__.py'>


tensor([1, 2, 3])

In [3]:
"""
#Goal: build a tool to create a connected structure of the model of nodes with a inbuilt gradient feature that tells each node
 how to update itself(take it's next step) to better fit the data.

Thoughts:
1. L = w*x + b; That's how loss functions are usually in some sense. We need to compute the gradient of L wrt w and b. x is the input... So, we are optimising the weights so that we have the least loss for the inputs of this form...
2. in the general case, we have change-needed: Delta = f(x, w1, w2, ... wn) where x is the input to that level/layer and w1, w2, ... wn are the tunable parameters in that level.
3. Our goal is to have a tool that can compute the gradient of Delta wrt w1, w2, ... wn will have terms from (x and wi's)
So, in this context, we need to know what L is, and then gradient of each level wrt to the next one(in the general case).
3. The idea is to reduce the LOSS, which means the layer just before needs to know how to change!
"""
import torch
import matplotlib.pyplot as plt
import numpy as np
import random

# Toy task: generate samples of y from x, where y = 3x + 4, and build a model to predict y from x.
# We will use a multi layer perceptron but without any activation function(we will add it later), and that model should generalize Universally!
def generate_linear_data(samples = 1, m = 3, c = 4, sd = 0.05):
    x = np.random.rand(samples)
    y = m*x + c + np.random.rand(samples)*sd
    return x, y

# We don't even many samples, let's build gradient descent with just 2 samples to uniquely identify m and c
x1 = 2; x2 = 5; m = 5; c = 7
# goal to gradient descent our way to find m and c, given x and y
random.seed(5)
mp = random.random(); cp = random.random()
lr = 0.02
x = x1 # let's say
y = m * x + c
yp = mp * x + cp
dyp_dm = x
dyp_dc = 1
L = (y - yp)**2
dL_dm = 2 * (y - yp) * (-dyp_dm)
dL_dc = 2 * (y - yp) * (-dyp_dc)
mp = mp - lr * dL_dm
cp = cp - lr * dL_dc

# Let's do this for 500 iterations
for epoch in range(501):
    dmp = 0; dcp = 0
    L = 0
    batch_update = False
    for x in [x1, x2]:
        y = m * x + c
        yp = mp * x + cp
        dyp_dm = x
        dyp_dc = 1
        dL_dm = 2 * (y - yp) * (-dyp_dm)
        dL_dc = 2 * (y - yp) * (-dyp_dc)
        if batch_update:
            dmp += - lr * dL_dm
            dcp += - lr * dL_dc
        else:
            mp -= lr * dL_dm
            cp -= lr * dL_dc
        L += (y - yp)**2
    if batch_update:
        mp += dmp
        cp += dcp
    L /= 2
    if epoch%100==0: print(f"Epoch: {epoch}, Loss: {L:.2f}, mLearnt: {mp:.2f}, cLearnt: {cp:.2f}")
print(f"Actual m: {m}, Actual c: {c}")
# We have proved that 2 examples are sufficient to uniquely identify m and c.
# Start with random m and c, and then gradient descent your way to find the actual m and c.

# Let's build useful tools for gradient descent, with pre-defined functions and operations...
# Take this step for example: dL_dc = 2 * (y - yp) * (-dyp_dc), let's give a structure to this!
# let's break down what is happening:
    # 1. yp = mp * x + cp
    # 2. L = (y - yp)**2
    # 3. Let l = y - yp
    # 4. L = l**2
    # 5. dL_dl = 2 * l; dl_dyp = -1; dyp_dmp = x; dyp_dc = 1
    # 6. dL_dmp = dL_dl * dl_dyp * dyp_dmp
    # 7. dL_dc = dL_dl * dl_dyp * dyp_dc
# what we have learnt is that to compute gradient at a level, information needs to flow from the L all the way to the level!
# Let's define dL_dx as grad_x
# L = l**2, grad_l = 2*l
# l = y - yp, grad_yp = grad_l*-1
# yp = mp*x + cp, grad_mp = grad_yp*x, grad_cp = grad_yp*1
# You see, that each level, the definition of the forward pass also gives the definition of the backward pass! Let's build a data structure to capture this!





Epoch: 0, Loss: 204.21, mLearnt: 6.04, cLearnt: 2.47
Epoch: 100, Loss: 0.28, mLearnt: 5.22, cLearnt: 5.88
Epoch: 200, Loss: 0.02, mLearnt: 5.05, cLearnt: 6.73
Epoch: 300, Loss: 0.00, mLearnt: 5.01, cLearnt: 6.93
Epoch: 400, Loss: 0.00, mLearnt: 5.00, cLearnt: 6.98
Epoch: 500, Loss: 0.00, mLearnt: 5.00, cLearnt: 7.00
Actual m: 5, Actual c: 7


In [11]:
"""
Thinking about the data structure :)
x -> forward pass -> y
define early as towards the x, and late as towards the y
1. Each intermediate node is formed out of an operation of atmost two(WLOG) earlier nodes say e1 and e2
2. We can now define the grad_e1 = grad_new(all later levels) * dnew_de1(this level); similarly for e2
3. We have to distinguish the X node to be non-tunable, similarly the structure should clearly know what the L node is...
4. We will call the backward() pass on the L node, and it should update all the nodes in the structure.

A point that I later realized is that a node can have multiple children... And each of them will have gradients...
So, the grad of the node will come from the sum of gradients through each of the children...


"""
from random import random

class Value:
    def __init__(self, val, children = None, op = None, grad = 0):
        self.val = val
        self.children = children
        self.op = op
        self.grad = grad
        self.backward = lambda: None

    def __repr__(self):
        return f"Value(data={self.val}, grad={self.grad})"
    
    # Construction of the nodes from elementary operations
    def __add__(self, other):
        other = Value(other) if not isinstance(other, Value) else other
        res = Value(self.val + other.val, children = [self, other], op = "+")
        def backward():
            # res = self + other                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            9
            # dL/dself = dL/dres * dres/dself 
            self.grad += res.grad
            other.grad += res.grad
            #print(f"self: {self}, other: {other}")
        res.backward = backward
        # every node should know the backward pass that needs to be invoked, when necessary!
        return res
    
    def __mul__(self, other):
        other = Value(other) if not isinstance(other, Value) else other
        res = Value(self.val * other.val, children = [self, other], op = "*")
        def backward():
            # in this step:
            # res = self * other
            # dL/dself = dL/dres * dres/dself ## dL/dres is easier to calculate as it is closer to L
            self.grad += res.grad * other.val # += because self can have other children too
            other.grad += res.grad * self.val
        res.backward = backward
        return res
    
    def __pow__(self, power):
        assert isinstance(power,(int, float)), "Only integer/float powers are supported"
        res = Value(self.val**power, children = [self], op = f"**{power}")
        def backward():
            # in this step: res = self**power
            # dL/dself = dL/dres * dres/dself
            self.grad += res.grad * power * self.val**(power-1)
        res.backward = backward
        return res
    
    # this backward pass when called will do the entire backpropagation starting from this node! dL/dnode calculation
    def back_prop(self):
        self.grad = 1 # dL/dL = 1
        # let us do a dfs topo sort to do the backpropagation
        topo = self.topo_sort()
        for node in topo:
            node.backward()
        
    def topo_sort(self):
        topo = []
        visited = set()
        def dfs(node):
            if node in visited: return
            if node.children:
                for child in node.children:
                    dfs(child)
            topo.append(node)
            visited.add(node)
        dfs(self)
        topo.reverse()
        return topo

    def step(self, lr):
        # set up the gradients and then update the values
        topo = self.topo_sort()
        self.back_prop()
        for node in topo:
            if node.op is not None:
                node.val -= lr * node.grad
                node.grad = 0 # reset the gradients for the next iteration

    def __rmul__(self, other):
        # cases like 2*Value(3)
        return self.__mul__(other)

    def __sub__(self, other):
        return self + (-1)*other

    def __neg__(self):
        # Eg: -Value(3)
        return -1*self # will work because __rmul__ is defined
    
    def __truediv__(self, other):
        return self * other**-1 # will work because __pow__ is defined
    
    def __rtruediv__(self, other):
        # other/self
        return other * self**-1
    
    def __radd__(self, other):
        # other + self
        return self + other
    
    def __rsub__(self, other):
        # other - self
        return -1*self + other
    
    # let us assume that cases like Val**Val are 2**Val are not supported for now...


In [14]:
# let's test the datastructure! 
"""    
What is it capable of? 
1. It can do a step to update it's weights through the network automatically to minimize the Loss...
2. Let's define the loss as: L = (y - (mx + c))**2
3. Let us initialise m and c randomly, and then do a step to update the weights to minimize the loss...
4. Let us fix the target m and c as 3 and 4. we are trying to get the line y = 3x+4
5. two sample points at x = 1, and at x = 2, y = 7, and y = 10, or let's say x is generated randomly at runtime :)
"""
def f(x): return 3*x + 4
from tqdm import tqdm
epochs = 1000
m = Value(random())
c = Value(random())
lr = 0.01
for i in tqdm(range(epochs)):
    x = random()
    y = f(x)
    L = (y - (m*x + c))**2
    L.step(lr*10)
    if i%100==0: print(f"Epoch: {i}, Loss: {L.val:.2f}, m: {m.val:.2f}, c: {c.val:.2f}")


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 16077.34it/s]

x: 0.9905314696261447, y: 6.971594408878434
Epoch: 0, Loss: 37.27, m: 0.10, c: 0.76
x: 0.7869303548644226, y: 6.360791064593268
x: 0.4899165051524964, y: 5.469749515457489
x: 0.20394966968968453, y: 4.611849009069053
x: 0.8400580028178369, y: 6.520174008453511
x: 0.1909897760393755, y: 4.572969328118127
x: 0.006661272992973033, y: 4.019983818978919
x: 0.47860125310167334, y: 5.435803759305021
x: 0.9227037840961261, y: 6.768111352288378
x: 0.9962001845975156, y: 6.988600553792547
x: 0.5136400464336393, y: 5.540920139300917
x: 0.040669317190074494, y: 4.1220079515702235
x: 0.46537975300276024, y: 5.396139259008281
x: 0.4381704448403164, y: 5.314511334520949
x: 0.6543013786064334, y: 5.962904135819301
x: 0.3770857338177367, y: 5.13125720145321
x: 0.4067208927500646, y: 5.220162678250194
x: 0.8549469749773994, y: 6.564840924932199
x: 0.7971750371360485, y: 6.3915251114081455
x: 0.25027169965715035, y: 4.750815098971451
x: 0.5449364464317669, y: 5.6348093392953
x: 0.04471488396893586, y: 4.